In [ ]:
from Cluster import Cluster
from Cassandra import Cassandra
from NoSQLBench import NoSQLBench, RunCommand

import enoslib as en
import enoslib.config as en_config

In [ ]:
en_config.set_config(g5k_cache=False, ansible_stdout="noop")

In [ ]:
cluster = Cluster(site="nancy",
                  cluster="gros",
                  walltime="0:30:00",
                  job_name="cassandra",
                  node_count=0,
                  seed_count=0,
                  client_count=1)

roles = cluster.get_resources()

In [ ]:
cassandra = Cassandra(name="cassandra", docker_image="adugois1/apache-cassandra-base:latest")

cassandra.setup(hosts=roles["cassandra"],
                seeds=roles["seeds"],
                not_seeds=roles["not_seeds"],
                conf_template="templates/cassandra-base.yaml")

cassandra.deploy_and_start()

In [ ]:
nb = NoSQLBench(name="nosqlbench", docker_image="nosqlbench/nosqlbench:nb5preview")

nb.setup(hosts=roles["clients"])

nb.deploy()

In [ ]:
nb.command(
    RunCommand.from_options(driver="stdout",
                            workload="/etc/nosqlbench/workloads/baseline",
                            tags="block:schema",
                            threads="1")
    .report_csv_to("/var/lib/nosqlbench")
)

In [ ]:
nb.sync_results(dest="./results-base")

In [ ]:
cassandra.cleanup()

In [ ]:
cluster.destroy()